<h1>Programming homework assignment Chapter 2 - Agents<h1/>

This base code is taken from https://github.com/aimacode/aima-python

This assignment's purpose is to experiment and understand all the techniques required to effectively program both performance-measuring environments and agents meant to traverse them and accomplish a set task.
The goal for this assignment is knowing which types of agents and agent functions are best to utilize in any environment, regardless if its known or unknown. 

In order to accomplish this goal, first we design a performance measuring vacuum-world environment. Initially the environment will only extend horizontally, we can easily change the size of this environment and dirt is randomly generated across the environment. Afterwards, we create a simple reflex agent which sucks if there is dirt present, then moves right or left, repeating this process until the room is clean. The performance measure is calculated by adding 10 points whenever the vacuum succesfully sucks dirt, and deducting a point when it moves left or right.

## Environments and Agents

In [1]:
from __future__ import annotations 
import random

#This class defines the simple vacuum agent. It takes as parameters the program to be used and its initial location
class VacuumAgent:

    def __init__(self, program = 'Reflex', initialLoc = (0, 0)) -> None:
        self.bump = False
        self.program = program
        self.performance = 0
        self.location = initialLoc
        #State only used for when programming is not reflex or random based
        self.map = [] 

    #The sensor available to the agent are accessed through this function
    def sense(self, env: VacuumEnvironment):
        if env.isClean(self.location):
            return 'Clean'
        else:
            return 'Dirty'

    #All valid moves are sensed with a given step size, NOT in use for this agent    
    def validMove(self, env: VacuumEnvironment, step = 1):
        moveloc = []
        for loc in env.locations:
            x = loc[0]
            y = loc[1]

            if(y == self.location[1]):
                if x == self.location[0] + step:
                    moveloc.append(loc)
                elif x == self.location[0] - step:
                    moveloc.append(loc)
                    
            elif(x == self.location[0]):
                if y == self.location[1] + step:
                    moveloc.append(loc)
                elif y == self.location[1] - step:
                    moveloc.append(loc)

        return moveloc
    
    #The actuators available to the agent are accessed through this function
    def actuate(self, action, env: VacuumEnvironment,):
        prevLoc = self.location

        if action == 'Right':
            self.location = (self.location[0] + 1, self.location[1])
            self.performance -= 1
        elif action == 'Left':
            self.location = (self.location[0] - 1, self.location[1])
            self.performance -= 1
        elif action == 'Up':
            self.location = (self.location[0], self.location[1] + 1)
            self.performance -= 1
        elif action == 'Down':
            self.location = (self.location[0], self.location[1] - 1)
            self.performance -= 1
        elif action == 'Suck':
            if env.isClean(self.location) == False:
                self.performance += 10
                env.cleanLocation(self, self.location)

        if(self.location not in env.locations):
            self.bump = True
            self.location = prevLoc

    #For the use of the state program of this agent. Adds a coordinate to the state list of invalid coordinates to rember not to use       
    def bumpListAppend(self, action):
        if action == 'Right':
            self.map.append((self.location[0] + 1, self.location[1]))

        elif action == 'Left':
            self.map.append((self.location[0] - 1, self.location[1]))

        elif action == 'Up':
            self.map.append((self.location[0], self.location[1] + 1))

        elif action == 'Down':
            self.map.append((self.location[0], self.location[1] - 1))

    #For the use of the state program of this agent. Checks if a given coordinate is in the bump list.
    def checkBumpList(self, location):
        if(location in self.map):
            return True
        else:
            return False

    #Given an action returns the resulting locationo of the agent
    def actionToLocation(self, action) -> tuple:
        if action == 'Right':
            return (self.location[0] + 1, self.location[1])

        elif action == 'Left':
            return (self.location[0] - 1, self.location[1])

        elif action == 'Up':
            return (self.location[0], self.location[1] + 1)

        elif action == 'Down':
            return (self.location[0], self.location[1] - 1)

    #Makes the agent take a step in the environment. Currently senses and takes an action.
    def act(self, env: VacuumEnvironment) -> str:
        if self.program == 'Reflex':

            actions = ['Right','Left','Suck']
            
            #Cleaning
            if self.sense(env) == 'Dirty':
                self.actuate(actions[2], env)
                return actions[2]

            #Movement
            elif self.sense(env) != 'Dirty': 
                if self.location in env.locations:
                    self.actuate(actions[0], env)
                    return actions[0]
                
            
        elif self.program == 'Random':
            #Sense current location
            #If dirty clean
            if self.sense(env) == 'Dirty':
                self.actuate('Suck', env)
                return 'Suck'

            #Random Action to change location
            else:
                actions = ['Right','Left','Up','Down']
                randomAction = random.choice(actions)
                self.actuate(randomAction, env)
                return randomAction
    
        elif self.program == 'State':
            #Sense current location
            #If dirty clean
            if self.sense(env) == 'Dirty':
                self.actuate('Suck', env)
                return 'Suck'

            #Random actions to change location
            #Check if coordinate is stored in state
            #If not, perform selected action
            #If location did not change add coordinate as invalid to the state

            validChoice  = False

            while(validChoice == False):
                actions = ['Right','Left','Up','Down']
                randomAction = random.choice(actions)
                if(self.checkBumpList(self.actionToLocation(randomAction)) == False):
                    self.actuate(randomAction, env)
                    validChoice = True

            if(self.bump == True):
                self.bumpListAppend(randomAction)
                self.bump =  False
            
            return randomAction

#Defines a modular environment for a vacuum agent. Parameters are width, height and a list with all dirty locations.
class VacuumEnvironment:
    def __init__(self, width, height, dirt = None) -> None:
        self.width = width
        self.height = height
        self.locations = []
        self.dirt = []
        self.genDirt()
        self.genLocations()

    #Generates a list of all valid locations within the environment
    def genLocations(self):
        for x in range(self.width):
            for y in range(self.height):
                self.locations.append((x,y))                 

    #Generates a list of dirty locations inside the environment
    def genDirt(self):
        for x in range(self.width):
            for y in range(self.height):
                if random.randint(0,1) == 1:
                    self.dirt.append((x,y))
    
    #Removes a location from the dirt list
    def cleanLocation(self, agent: VacuumAgent, loc):
        if loc in self.dirt:
            self.dirt.remove(loc)

    #Checks the dirt/clean state of a given location
    def isClean(self, loc):
        if loc in self.dirt:
            return False
        else:
            return True



In [2]:
#Reflex Test in a horizontal environment

reflexEnv = VacuumEnvironment(20, 1)
print(f'Environment is created')
print()

#Number of actions made by the agent
limit = 50
actions = 0
reflexAgent = VacuumAgent('Reflex')

while(reflexEnv.dirt and actions < limit):
    actions = actions + 1
    reflexAgent.act(reflexEnv)


print(f'Reflex Agent performance after {actions} actions is: {reflexAgent.performance}')

Environment is created

Reflex Agent performance after 26 actions is: 51


Modifying and expanding the previous design, its no longer just horizontal and the shape, size, vacuum, and dirt placement of the environment is now unknown. Therefore the agent now also must be able to travel up or down in order to traverse and clean the environment.

In [7]:
#Testing Reflex Agent in a unknown environment that expands horizontally and vertically

reflexEnv = VacuumEnvironment(2, 2)
print(f'A 2x2 Environment is created')
print()

#Number of actions made by the agent
limit = 50
actions = 0
reflexAgent = VacuumAgent('Reflex')

while(reflexEnv.dirt and actions < limit):
    actions = actions + 1
    reflexAgent.act(reflexEnv)

print(f'Reflex Agent performance after {actions} actions is: {reflexAgent.performance}')

A 2x2 Environment is created

Reflex Agent performance after 50 actions is: -28


Utilizing a simple reflex agent is perfectly rational in this environment since it correctly perceives its location and if it's dirty, gets awarded points for cleaning dirt, loses points for moving unnecessarily, and it only has 5 available actions, being "Left", "Right", "Up", "Down, and "Suck".

If a simple reflex agent were to have a randomized function to determine where it moves, it has a chance to beat a simple reflex agent if it were to randomly choose the best options every single time. Also, a simple reflex agent can get stuck which can be solved by a random agent function. The random agent function can get the agent “unstuck”.

In [8]:
#Random Reflex Test
def randomTest():
    x = random.randint(1,5)
    y = random.randint(1,5)
    print(f'Creating environment of size (',x,',',y,')')
    randomEnv = VacuumEnvironment(x, y)
    print(f'Environment is created')
    print()

    #Number of actions made by the agent
    limit = 50
    actions = 0
    randomAgent = VacuumAgent('Random')

    while(randomEnv.dirt and actions < limit):
        actions = actions + 1
        randomAgent.act(randomEnv)

    print(f'Random Reflex Agent performance after {actions} actions is: {randomAgent.performance}')
    print()

testNumber = 1
while(testNumber <= 3):
    print('Random Test #: ', testNumber)
    testNumber += 1
    randomTest()

Random Test #:  1
Creating environment of size ( 4 , 3 )
Environment is created

Random Reflex Agent performance after 13 actions is: 42

Random Test #:  2
Creating environment of size ( 5 , 5 )
Environment is created

Random Reflex Agent performance after 50 actions is: 5

Random Test #:  3
Creating environment of size ( 4 , 2 )
Environment is created

Random Reflex Agent performance after 4 actions is: 7



However, in a sufficiently large environment, a randomized function simple reflex agent outperforming an ordinary simple reflex agent is very improbable, especially if its designed for the reflex agent. Testing such an environment

In [5]:
#Random Reflex Test Large Environment
import copy
randomEnv = VacuumEnvironment(20,1)
reflexEnv = copy.deepcopy(randomEnv)
print(f'Environment is created')
print()

#Number of actions made by the agent
limit = 50
actions = 0
randomAgent = VacuumAgent('Random')

while(randomEnv.dirt and actions < limit):
    actions = actions + 1
    randomAgent.act(randomEnv)
print(f'Random Reflex Agent performance after {actions} actions is: {randomAgent.performance}')

#Reflex Agent in the same environment
limit = 50
actions = 0
reflexAgent = VacuumAgent('Reflex')

while(randomEnv.dirt and actions < limit):
    actions = actions + 1
    reflexAgent.act(reflexEnv)
print(f'Reflex Agent performance after {actions} actions is: {reflexAgent.performance}')



Environment is created

Random Reflex Agent performance after 50 actions is: -6
Reflex Agent performance after 50 actions is: 38


The randomized agent performs poorly in 1-dimensional environments because two of its actions (moving up and down) are not valid and affect the performance.

Giving a reflex agent a state.

In [14]:
#State Reflex Test

#Rand size
width, height = random.randint(1,100), random.randint(1,100)

#Rand init location
initloc = (random.randint(0,100), random.randint(0,100))

stateEnv = VacuumEnvironment(width, height)
print(f'(',width,',',height,') State environment is created')
print()

#Number of actions made by the agent
limit = 1000
actions = 0
stateAgent = VacuumAgent('State')

while(stateEnv.dirt and actions < limit):
    actions = actions + 1
    stateAgent.act(stateEnv)

print(f'State Reflex Agent performance after {actions} actions is: {stateAgent.performance}')

( 75 , 66 ) State environment is created

State Reflex Agent performance after 1000 actions is: 936


This reflex agent with state was designed to have a memory of all locations that the agent cannot move to, thus reducing the amount of invalid movements the agent attempts. This agent can outperform the simple reflex agent in environments with invalid moves that the reflex agent can get stuck in. We can see that its performance is greater than the reflex agent without a state, even in larger environments.

In conclusion, we have learned that programming environments and intelligent agents that are able to traverse them can be very challenging at first, and that different agents can perform differently across different environments.